In [ ]:
!pip install tflearn

Import file

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

Load Data

In [4]:
from google.colab import files
files.upload()

Saving data.json to data.json


{'data.json': b'{"intents": [\r\n        {"tag": "greeting",\r\n         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],\r\n         "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "goodbye",\r\n         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day"],\r\n         "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "age",\r\n         "patterns": ["how old", "how old is tim", "what is your age", "how old are you", "age?"],\r\n         "responses": ["I am 18 years old!", "18 years young!"],\r\n         "context_set": ""\r\n        },\r\n        {"tag": "name",\r\n         "patterns": ["what is your name", "what should I call you", "whats your name?"],\r\n         "responses": ["You can call me Tim.", "I\'m Tim!", "I\'m Tim aka Tech With Ti

In [ ]:
with open("data.json") as file:
    data = json.load(file)

Sentence stemming

In [ ]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
  #Extracting Data
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    #convert our training data and output to numpy arrays
    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [9]:
#tensorflow.reset_default_graph()
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

#save model
model.save("model.tflearn")
#load model
model.load("model.tflearn")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


Predction

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Talk to you later
You: whats your name?
Sad to see you go :(
You: age
I am 18 years old!
You: name
Good to see you again!
You: fuck
18 years young!
You: food
18 years young!
You: bye
18 years young!
You: thank you
I'm Tim!
You: what`s your name
18 years young!
You: ?
18 years young!
You: hello
We are open 7am-4pm Monday-Friday!
